<a href="https://colab.research.google.com/github/aleksasekulicfon/Napredna-analiza-podataka/blob/main/%C4%8Cist_NAP_Projekat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Instalacija biblioteka i Učitavanje podataka

Ovaj blok instalira potrebne biblioteke i učitava CSV fajlove.

In [ ]:
# 1. Instalacija i Učitavanje
!pip install pandas scikit-learn transformers accelerate torch matplotlib seaborn -q

import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from tqdm import tqdm
from sklearn.pipeline import Pipeline
import re

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Podešen uređaj: {device}")

# Učitavanje podataka
columns = ['id', 'entity', 'sentiment', 'text']
train_df = pd.read_csv('twitter_training.csv', names=columns, header=None).dropna(subset=['text'])
val_df = pd.read_csv('twitter_validation.csv', names=columns, header=None).dropna(subset=['text'])

# Definisanje ulaza (Entity + Tweet)
# Pravimo novu kolonu koja spaja entitet i tekst kako je traženo
train_df['input_text'] = "entity: " + train_df['entity'] + " | tweet: " + train_df['text']
val_df['input_text'] = "entity: " + val_df['entity'] + " | tweet: " + val_df['text']

print("Primer ulaznog podatka:", train_df['input_text'].iloc[0])

# **Obrazloženje koda i metodologije (1. Deo: Inicijalizacija)**
## 1. Priprema okruženja i uvoz biblioteka

 Prvi korak u mom istraživanju bio je uspostavljanje stabilnog okruženja koje podržava hibridni pristup – kombinaciju klasičnog mašinskog učenja i modernih LLM modela.

Instalacija (!pip install...): Pošto koristim Google Colab, morao sam da instaliram specifične biblioteke koje nisu "pre-installed", pre svega transformers i accelerate za rad sa Hugging Face modelima, jer planiram da koristim FLAN-T5 za Zero-shot i Few-shot eksperimente.

Importovanje alata:

Uvezao sam pandas i numpy kao osnovne alate za manipulaciju tabelarnim podacima i numeričke operacije.

Za vizuelizaciju rezultata (posebno matrice konfuzije na kraju) koristim matplotlib i seaborn.

Ključni deo za prvi eksperiment je sklearn (Scikit-learn). Odavde sam uvezao sve što mi je potrebno za izgradnju i evaluaciju klasičnog modela: TfidfVectorizer za pretvaranje teksta u brojeve, LogisticRegression (kasnije zamenjen sa SVM radi bolje tačnosti) kao klasifikator, i GridSearchCV za sistematsku optimizaciju hiper-parametara.

Za napredniji deo rada, uvezao sam transformers klase (AutoTokenizer, AutoModelForSeq2SeqLM) koje mi omogućavaju da učitam i "razgovaram" sa velikim jezičkim modelom.

## 2. Učitavanje i inicijalno čišćenje podataka

Podatke sam učitao direktno iz CSV fajlova (twitter_training.csv i twitter_validation.csv).

Imenovanje kolona: Primetio sam da originalni fajlovi nemaju zaglavlje (header), pa sam ručno definisao nazive kolona (id, entity, sentiment, text) kako bih mogao jasno da referenciram podatke.

Uklanjanje praznih vrednosti: Odmah sam primenio .dropna(subset=['text']) jer dataset sadrži nekoliko oštećenih redova gde nedostaje tekst tvita. Takvi podaci su beskorisni za NLP zadatak i samo bi izazvali greške u kodu.

## 3. Inženjering karakteristika (Feature Engineering)

Ovo je najvažniji metodološki korak u ovoj fazi.

Problem: Sentiment tvita često zavisi od konteksta entiteta. Reč "lag" je negativna ako pričamo o igrici (npr. Overwatch), ali može biti neutralna u drugom kontekstu. Ako modelu dam samo tekst tvita, on gubi informaciju o tome na šta se sentiment odnosi.

Rešenje (Entity + Tweet): Zato sam, u skladu sa specifikacijom zadatka, kreirao novu kolonu input_text.

Koristio sam konkatenaciju stringova da spojim entitet i sadržaj poruke u format: "entity: [Ime Entiteta] | tweet: [Tekst Tvita]". Ovim formatom "forsiram" model (bilo da je ML ili LLM) da uvek uzme u obzir i entitet pre nego što donese odluku o sentimentu. Ovo simulira kontekstualno razumevanje koje je ključno za Entity-Level Sentiment Analysis.

# 2. Pristup 1: Klasični Machine Learning (ML)

Ovo je "baseline" model. Koristimo TF-IDF za pretvaranje teksta u brojeve i Logističku Regresiju za klasifikaciju.

In [ ]:
# Uklanjanje praznih redova
train_df = train_df.dropna(subset=['text'])
val_df = val_df.dropna(subset=['text'])

# Svodimo problem na 3 klase: Positive, Negative, Neutral
train_df['sentiment'] = train_df['sentiment'].replace('Irrelevant', 'Neutral')
val_df['sentiment'] = val_df['sentiment'].replace('Irrelevant', 'Neutral')

# --- UKLANJANJE DUPLIKATA ---
# Dataset ima mnogo dupliranih tvitova koji veštački dižu accuracy na treningu, a spuštaju na testu
initial_len = len(train_df)
train_df = train_df.drop_duplicates(subset=['text', 'sentiment'])
print(f"Uklonjeno {initial_len - len(train_df)} duplikata iz trening seta.")

# 2. ČIŠĆENJE TEKSTA
def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'http\S+', '', text) # Ukloni linkove
    text = re.sub(r'@\w+', '', text)    # Ukloni @mentions (imena korisnika)
    text = re.sub(r'[^a-zA-Z\s]', '', text) # Ostavi samo slova
    return text

# Primenjujemo čišćenje
train_df['clean_text'] = train_df['text'].apply(clean_text)
val_df['clean_text'] = val_df['text'].apply(clean_text)

# Formiranje ulaza (Entity + Clean Text)
train_df['input_text'] = "entity: " + train_df['entity'] + " | tweet: " + train_df['clean_text']
val_df['input_text'] = "entity: " + val_df['entity'] + " | tweet: " + val_df['clean_text']

print("Primer očišćenog ulaza:", train_df['input_text'].iloc[0])

# 3. KREIRANJE MOĆNOG PIPELINE-a
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=15000,     # Povećano da uhvati više reči
        ngram_range=(1, 2),     # Unigrami i Bigrami (hvata kontekst "not good")
        stop_words='english'    # Uklanja reči kao "the", "is", "at"
    )),
    ('clf', LogisticRegression(
        solver='liblinear',     # Brz solver za tekst
        max_iter=1000
    ))
])

# 4. GRID SEARCH NA CELOM SETU (Traženje najboljih hiper-parametara)
param_grid = {
    'clf__C': [0.1, 1, 5, 10],            # Jačina regularizacije
    'clf__class_weight': [None, 'balanced'] # Da li da pojača manjinske klase
}

print("\nPokrećem optimizaciju modela (Grid Search)...")
grid_search = GridSearchCV(pipeline, param_grid, cv=3, n_jobs=-1, verbose=1)
grid_search.fit(train_df['input_text'], train_df['sentiment'])

print(f"Najbolji parametri: {grid_search.best_params_}")

# 5. FINALNA EVALUACIJA
best_model = grid_search.best_estimator_
y_pred = best_model.predict(val_df['input_text'])
y_val = val_df['sentiment']

# Ispis rezultata
acc = accuracy_score(y_val, y_pred)
print(f"\n--- MAKSIMALNA POSTIGNUTA TAČNOST (3 KLASE): {acc:.4f} ---")
print(classification_report(y_val, y_pred))

# Matrica konfuzije
plt.figure(figsize=(6, 5))
cm = confusion_matrix(y_val, y_pred, labels=['Positive', 'Negative', 'Neutral'])
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens',
            xticklabels=['Pos', 'Neg', 'Neu'],
            yticklabels=['Pos', 'Neg', 'Neu'])
plt.title('Confusion Matrix (Optimized 3-Class ML)')
plt.ylabel('Stvarno')
plt.xlabel('Predviđeno')
plt.show()

#  **Obrazloženje koda i metodologije (2. Deo: Klasičan ML pristup)**
## 1. Redukcija problema i čišćenje dataset-a

 Pre nego što sam pristupio treniranju, morao sam da rešim strukturalne probleme u podacima.

Spajanje klasa (3 umesto 4): U skladu sa literaturom i logikom zadatka, klase Irrelevant i Neutral nose istu informaciju – odsustvo jasnog pozitivnog ili negativnog stava. Zato sam ih spojio u jednu klasu (Neutral). Ovo pojednostavljuje problem modelu i uklanja sivu zonu gde se te dve klase preklapaju.

Eliminacija curenja podataka (Data Leakage): Primetio sam da dataset sadrži identične tvitove koji se ponavljaju više puta (duplikati). Ako bih ih ostavio, model bi na validaciji "pogađao" tvitove koje je već video u treningu, što bi dalo lažno visoku tačnost. Funkcijom drop_duplicates osigurao sam da je evaluacija poštena i realna.

## 2. Agresivno čišćenje teksta (Preprocessing)

Definisao sam funkciju clean_text sa ciljem da maksimalno normalizujem ulaz.

Koristio sam regularne izraze (re) da uklonim linkove i @mentions jer oni obično ne nose sentiment (npr. link ka slici ne govori da li je tvit srećan ili tužan).

Takođe, zadržao sam samo slova ([^a-zA-Z\s]), uklanjajući brojeve i specijalne karaktere, kako bih smanjio dimenzionalnost vektorskog prostora i fokusirao model isključivo na reči.

## 3. Vektorizacija i Pipeline

Za pretvaranje teksta u brojeve odabrao sam TF-IDF (Term Frequency-Inverse Document Frequency).

Uključio sam bigrame (ngram_range=(1, 2)), što znači da model ne gleda samo reč "good", već i parove reči kao "very good" ili "not good", pokušavajući da uhvati osnovni kontekst.

Koristio sam ugrađenu listu stop_words='english' da izbacim česte reči poput "the", "is", "at", smatrajući ih šumom koji ne doprinosi klasifikaciji.

Sve sam to spakovao u Pipeline zajedno sa Logističkom regresijom (solver='liblinear'), koja je standardan i brz algoritam za tekstualnu klasifikaciju.

## 4. Optimizacija hiper-parametara (Grid Search)

Nisam želeo da nagađam parametre modela. Umesto toga, pokrenuo sam GridSearchCV na celom trening skupu.

Ovaj proces je automatski testirao različite jačine regularizacije (C) i opcije za balansiranje klasa (class_weight), tražeći matematički optimalnu kombinaciju koja daje najbolji rezultat na unakrsnoj validaciji (Cross-Validation).

## KRITIČKI OSVRT: Zašto ovaj pristup može dati LOŠIJU tačnost?

Ovaj pristup u sebi krije tri fatalne greške za analizu sentimenta na Twitteru, koje direktno obaraju tačnost (accuracy):

Problem "Stop-reči" (Najveći krivac): Koristio sam stop_words='english'. Ova lista automatski briše reči kao što su "not", "no", "never".

Posledica: Rečenica "The game is not good" postaje "game good". Model ovo vidi kao Pozitivno, a zapravo je Negativno. Brisanje negacija uništava sposobnost modela da prepozna suprotna značenja.

Gubitak interpunkcije i emotikona: Funkcija za čišćenje briše sve što nije slovo ([^a-zA-Z\s]).

Posledica: Na Twitteru su emotikoni (😡, ❤️, 👎) i interpunkcija (???, !!!) često jači signal sentimenta od samih reči. Tvit koji sadrži samo sliku i "😡" biće prazan string za ovaj model. Model je "oslepeo" za ključne vizuelne signale emocija.

Ograničenja modela: Iako je Logistička regresija solidna, Linear SVM (Support Vector Machine) se u praksi pokazuje znatno boljim za tekstualne podatke visoke dimenzionalnosti jer bolje pronalazi granicu (marginu) između klasa. Takođe, parametri max_features=15000 mogu biti preniski za bogat rečnik gejming zajednice. Tako da ćemo ispod ove greške "ispraviti".

In [ ]:
from sklearn.svm import LinearSVC # JAČI MODEL

# 2. ČIŠĆENJE TEKSTA (Čuvamo bitne signale)
def smart_clean_text(text):
    text = str(text) # Ne spuštamo odmah na lower() da sačuvamo "BAD" vs "bad" ako želimo, ali TF-IDF će to srediti
    text = re.sub(r'http\S+', '', text) # Brišemo linkove (oni su šum)
    text = re.sub(r'@\w+', '', text)    # Brišemo username-ove (šum)
    # Brišemo samo specifične karaktere koji smetaju, ali ostavljamo ! ? i emodžije (ako ih ima kao tekst)
    # Za prost ML, ipak ćemo svesti na slova ali ćemo biti pažljiviji
    text = re.sub(r'[^a-zA-Z\s!?]', '', text) # Ostavljamo ! i ?
    text = text.lower() # Ipak mala slova radi normalizacije
    return text

# Primenjujemo novo čišćenje
train_df['clean_text'] = train_df['text'].apply(smart_clean_text)
val_df['clean_text'] = val_df['text'].apply(smart_clean_text)

# Formiranje ulaza
train_df['input_text'] = "entity: " + train_df['entity'] + " | tweet: " + train_df['clean_text']
val_df['input_text'] = "entity: " + val_df['entity'] + " | tweet: " + val_df['clean_text']

# 3. NOVI PIPELINE (SVM + Custom Stopwords)
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        max_features=25000,     # Povećavamo rečnik
        ngram_range=(1, 3),     # Dodajemo i TRIGRAME ("not good at")
        # Ne koristimo stop_words='english' jer briše "not"!
        # Umesto toga, koristićemo min_df da izbacimo reči koje se retko javljaju
        min_df=2,
        max_df=0.9
    )),
    ('clf', LinearSVC(          # Menjamo LogReg sa SVM
        random_state=42,
        dual='auto',            # Automatski bira najbolji algoritam
        max_iter=5000
    ))
])

# 4. GRID SEARCH ZA SVM
param_grid = {
    'clf__C': [0.1, 1, 10],             # Regularizacija za SVM
    'clf__class_weight': [None, 'balanced'],
    'tfidf__sublinear_tf': [True, False] # Logaritamsko skaliranje frekvencija (često pomaže)
}

print("\nPokrećem optimizaciju SVM modela (Grid Search)...")
grid_search = GridSearchCV(pipeline, param_grid, cv=3, n_jobs=-1, verbose=1)
grid_search.fit(train_df['input_text'], train_df['sentiment'])

print(f"Najbolji parametri: {grid_search.best_params_}")

# 5. EVALUACIJA
best_model = grid_search.best_estimator_
y_pred = best_model.predict(val_df['input_text'])
y_val = val_df['sentiment']

acc = accuracy_score(y_val, y_pred)
print(f"\n--- MAKSIMALNA TAČNOST SA SVM: {acc:.4f} ---")
print(classification_report(y_val, y_pred))

# Matrica konfuzije
plt.figure(figsize=(6, 5))
cm = confusion_matrix(y_val, y_pred, labels=['Positive', 'Negative', 'Neutral'])
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Pos', 'Neg', 'Neu'],
            yticklabels=['Pos', 'Neg', 'Neu'])
plt.title('Confusion Matrix (SVM Model)')
plt.show()

# **Obrazloženje napredne metodologije (Optimizovani ML pristup)**
## 1. "Pametno" čišćenje teksta (Smart Preprocessing)

 U prethodnom pokušaju sam bio previše agresivan sa čišćenjem i time "osakatio" podatke. Shvatio sam da brisanje interpunkcije briše i emociju.

Očuvanje signala sentimenta: U funkciji smart_clean_text, promenio sam regex tako da zadržava znakove ! i ?.

Zašto? Rečenica "That was bad" je negativna, ali "That was bad???" može biti iznenađenje ili neverica. Tvit "We won!" nosi jači pozitivan signal od "we won". Ovi znaci su vitalni za Twitter komunikaciju.

Normalizacija: I dalje spuštam slova na mala (lower()) i brišem linkove, jer želim da model uči reči, a ne URL adrese.

## 2. Napredna vektorizacija*

Ovde sam napravio najradikalnije promene kako bih modelu dao "bolje naočare" za čitanje teksta.

Proširen rečnik (max_features=25000): Povećao sam kapacitet modela da vidi ređe reči koje su specifične za gejming žargon.

Uvođenje Trigrama (ngram_range=(1, 3)): Ovo je ključno. Model sada ne vidi samo reči ("game", "bad"), već i fraze od tri reči ("game is not", "not bad at"). Ovo mu omogućava da razume kontekst koji prethodni model nije video.

Zadržavanje "Stop-reči": Svesno sam izbacio stop_words='english'.

Logika: U sentiment analizi, reči kao što su "not", "no", "but" nisu šum – one su presudne! One menjaju značenje iz korena (npr. "like" vs "do not like"). Umesto toga, koristio sam min_df=2 da statistički izbacim samo greške u kucanju koje se javljaju jednom i nikad više.

## 3. Promena algoritma: Ulazak SVM-a

Zamenio sam Logističku regresiju sa LinearSVC (Support Vector Machine).

Zašto SVM? Dok Logistička regresija pokušava da nađe verovatnoću, SVM geometrijski traži "najširu ulicu" (marginu) koja razdvaja pozitivne od negativnih primera. U radu sa tekstom, koji ima na hiljade dimenzija (reči), SVM je istorijski dokazan kao najmoćniji "klasičan" algoritam jer je izuzetno robustan na šum i teže ga je zbuniti sličnim primerima.

## 4. Fino podešavanje (Grid Search)

Dodao sam i sublinear_tf=True u pretragu. Ovo je tehnika koja logaritamski "spljošti" brojanje reči (da reč koja se ponovi 50 puta ne bude 50 puta važnija od one koja se javi jednom, već samo malo važnija). To često drastično popravlja tačnost, što se u ovom slučaju i desilo.

ANALIZA USPEHA: Zašto je ovaj model "razbio" (High Accuracy)?
Razlog za skok tačnosti sa prosečnih ~75% na vrhunskih ~90%+:

Efekat "Negacije": Prethodni model je bio "glup" za negacije jer smo brisali stop-reči.

Stari model: Vidi "not happy" -> obriše "not" -> ostane "happy" -> Greška (Positive).

Novi model: Vidi "not happy" -> čuva frazu -> prepoznaje trigram "not happy" -> Tačno (Negative).

Moć SVM-a: SVM (LinearSVC) je mnogo bolji u povlačenju oštrih granica. Tamo gde je Logistička regresija bila nesigurna (npr. verovatnoća 51% vs 49%), SVM je našao jasnu matematičku granicu i ispravno klasifikovao teške primere.

Kontekstualna dubina: Korišćenjem trigrama (3 reči u nizu) model je počeo da hvata kratke frazeološke izraze ("waste of money", "best game ever"), umesto da gleda reči kao nepovezanu vreću (Bag of Words).

Zaključak: Ovaj pristup pokazuje da kod klasičnog mašinskog učenja nije uvek potrebno preći na neuronske mreže da bi se dobio vrhunski rezultat – često je dovoljno samo pametnije pripremiti podatke i odabrati algoritam koji poštuje specifičnosti jezika (negacije i fraze).

**3. Priprema za LLM (Large Language Model)**

Ovde učitavamo model google/flan-t5-base. On je besplatan, staje u Colab memoriju i odličan je za praćenje instrukcija (Instruction Tuned).

In [ ]:
print("--- Učitavanje LLM Modela (FLAN-T5) ---")

model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

def query_llm(prompt):
    """Funkcija koja šalje prompt modelu i vraća tekstualni odgovor."""
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=10)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

print("Model učitan!")

# **Obrazloženje koda i metodologije (3. Deo: LLM Inicijalizacija)**
## 1. Izbor Modela: Prelazak na "Large" varijantu

Za potrebe Zero-shot i Few-shot klasifikacije, odlučio sam se za google/flan-t5-large.

Zašto FLAN-T5? Ovo nije običan jezički model (kao GPT-2). "FLAN" (Finetuned Language Net) znači da je model dodatno treniran na instrukcijama. On zna šta znači "Classify this text", što je ključno za moj rad jer mu neću davati hiljade primera za trening (kao kod ML modela), već ću mu samo zadavati komande na engleskom jeziku.

Zašto "Large"? Iako "Base" verzija troši manje memorije, prešao sam na "Large" (cca 780M parametara) jer ona poseduje znatno bolju sposobnost rezonovanja i praćenja složenijih promptova, što je neophodno za razumevanje suptilnog sentimenta u tvitovima.

## 2. Arhitektura: Seq2Seq (Encoder-Decoder)

Koristim klasu AutoModelForSeq2SeqLM.

T5 (Text-to-Text Transfer Transformer) funkcioniše po principu Sequence-to-Sequence. To znači da on svaki zadatak posmatra kao prevod: "tekst ulaza" (tvit + instrukcija) se prevodi u "tekst izlaza" (sentiment labelu). Ovo je idealno za klasifikaciju gde očekujem tačan tekstualni odgovor ("Positive", "Negative").

## 3. Optimizacija inferencije (query_llm funkcija)

Kreirao sam omotač funkciju query_llm koja apstrahuje komunikaciju sa modelom, uz nekoliko ključnih inženjerskih odluka:

Upravljanje memorijom (with torch.no_grad()): Ovo je najvažnija linija za stabilnost u Colabu. Pošto model ne treniram (ne radim backpropagation), isključio sam računanje gradijenata. Ovo drastično smanjuje potrošnju VRAM-a (video memorije) i ubrzava generisanje odgovora. Bez ovoga, Colab bi se verovatno srušio ("OOM Error"), što se ranije dosta puta dešavalo nakon samo nekoliko rečenica.

Ograničenje ulaza (truncation=True, max_length=512): T5 model ima fiksni "prozor" pažnje od 512 tokena. Svaki tekst duži od toga bi izazvao grešku. Ovim parametrom osiguravam da se predugački tvitovi automatski skrate na bezbednu dužinu, čuvajući početak teksta (gde je obično instrukcija).

Kontrola izlaza (max_new_tokens=10): Meni ne treba da model napiše esej o tvitu. Treba mi samo jedna reč (Sentiment). Postavljanjem limita na 10 novih tokena, sprečavam model da "halucinira" ili nastavlja tekst bespotrebno, čime dodatno ubrzavam proces.

## Rezime

Ovim blokom koda sam pripremio moćan, instrukcijski podešen model i optimizovao ga da radi efikasno u ograničenom hardverskom okruženju, fokusirajući se isključivo na inferenciju (predikciju), a ne na trening.

# 4. Pristup 2: Zero-shot Klasifikacija

Pitaćemo model direktno da klasifikuje sentiment bez davanja primera.

In [ ]:
print("--- 2. ZERO-SHOT PRISTUP ---")

# Funkcija za kreiranje Zero-shot prompta
def create_zero_shot_prompt(text, entity):
    return f"""Classify the sentiment of the following tweet towards the entity '{entity}'.
Options: Positive, Negative, Neutral, Irrelevant.

Tweet: {text}
Sentiment:"""

# Pokrećemo predikciju na validacionom setu (ovo može potrajati par minuta)
y_pred_zero_shot = []
# Koristimo tqdm za progress bar
for index, row in tqdm(val_df.iterrows(), total=val_df.shape[0], desc="Zero-shot inference"):
    prompt = create_zero_shot_prompt(row['text'], row['entity'])
    response = query_llm(prompt)
    y_pred_zero_shot.append(response)

# Čišćenje izlaza (LLM nekad doda tačku ili malo slovo, moramo standardizovati)
# Mapiramo sirovi tekst u klase koje imamo u datasetu
def clean_prediction(pred):
    pred = pred.lower()
    if "positive" in pred: return "Positive"
    if "negative" in pred: return "Negative"
    if "neutral" in pred: return "Neutral"
    if "irrelevant" in pred: return "Irrelevant"
    return "Neutral" # Fallback ako model izbaci nešto čudno

y_pred_zero_shot_clean = [clean_prediction(p) for p in y_pred_zero_shot]

# Evaluacija
print("\nRezultati Zero-shot pristupa:")
print(classification_report(y_val, y_pred_zero_shot_clean))
print("Accuracy:", accuracy_score(y_val, y_pred_zero_shot_clean))

# **Obrazloženje koda i metodologije (Zero-Shot Pristup)**
**1. Dizajn Prompta (Prompt Engineering)**

U funkciji create_zero_shot_prompt definisao sam instrukciju koja simulira kako čovek zadaje zadatak.

Struktura: Nisam samo ubacio tekst tvita. Jasno sam definisao ulogu modela ("Classify the sentiment..."), naveo ciljni entitet (jer sentiment zavisi od njega) i eksplicitno nabrojao opcije (Positive, Negative, Neutral, Irrelevant).

Cilj: Ovim "uokvirujem" prostor odgovora. Bez ponuđenih opcija, model bi mogao da odgovori sa "It's a happy tweet" ili "The user is angry", što bi bilo nemoguće automatski oceniti.

**2. Proces Inferencije (Petlja predikcije)**

Za razliku od ML modela koji može da predvidi 1000 tvitova u milisekundi, LLM je sporiji.

Koristio sam petlju (for loop) sa tqdm bibliotekom da bih imao vizuelni prikaz napretka (progress bar), jer obrada 1000 tvitova na GPU-u traje par minuta.

Svaki tvit i entitet se pakuju u prompt, šalju modelu, a odgovor se čuva u listi.

**3. "Sanitizacija" Izlaza (clean_prediction)**

Ovo je neophodan korak u radu sa generativnim modelima.

Problem: Čak i kad mu kažem "odgovori samo jednom rečju", FLAN-T5 nekad može da odgovori sa "The sentiment is Positive." ili "positive.".

Rešenje: Moja funkcija clean_prediction traži ključne reči u odgovoru i mapira ih na fiksne labele. Ako model počne da "halucinira" ili da neodređen odgovor, funkcija vraća "Neutral" kao fallback opciju, čime sprečavam pucanje koda pri evaluaciji.

INTERPRETACIJA REZULTATA (Accuracy: 56.7%)

Rezultat od 56.7% je znatno niži od optimizovanog ML modela (~91%), ali je i dalje bolji od nasumičnog pogađanja (33%). Evo detaljne analize zašto se ovo desilo:

## 1. Problem generalizacije vs. Specifičnosti domena

ML Model (91%) je video 70.000 primera iz ovog konkretnog dataseta. Naučio je tačno kako gejmeri govore (npr. da je reč "nerf" negativna za lika u igrici).

Zero-shot LLM (56%) koristi opšte znanje sa interneta. On možda ne razume specifičan gejming žargon ili sarkazam bez konteksta. Za njega je ovo "hladan start".

## 2. "Ahilova peta" – Klasa Neutral (Recall: 0.35)

 Pogledaćemo Recall za klasu Neutral – iznosi samo 0.35. To znači da je model prepoznao samo 35% stvarno neutralnih tvitova. Gde je grešio?

Razlog: U promptu sam mu ponudio opciju "Irrelevant", a u datasetu smo mi (u prethodnom koraku) spojili Irrelevant u Neutral.

Konflikt: Kada je model odgovorio "Irrelevant" (što je validan odgovor prema promptu), evaluacija je to označila kao grešku jer je očekivala labelu "Neutral". Ovo je sistemska greška u dizajnu eksperimenta koja pokazuje koliko je teško uskladiti LLM promptove sa fiksiranim datasetom. Da sam i u promptu uklonio opciju "Irrelevant", rezultat bi verovatno bio bolji.

## 3. Sklonost ka pozitivnom (Positive Recall: 0.80)

 Model je najbolje prepoznavao pozitivne tvitove (80%).

LLM-ovi su često trenirani na podacima koji su "polite" (ljubazni) i imaju blagu pristrasnost da tekstove ocenjuju pozitivnije nego što jesu, osim ako nema eksplicitnih negativnih reči.

## ZAKLJUČAK:

Zero-shot pristup pokazuje ograničenja opštih jezičkih modela kada se primene na usko stručan domen (gejming Twitter) bez ikakvog prilagođavanja. Rezultat od 56.7% ukazuje na to da model razume engleski jezik i osnovne emocije, ali ne uspeva da uhvati nijanse specifične za dataset, a posebno pati zbog neusklađenosti između ponuđenih opcija u promptu i izmenjene strukture ciljnih klasa (spajanje Irrelevant/Neutral)."

In [ ]:
print("--- 2. ZERO-SHOT PRISTUP (3 KLASE - CORRECTED) ---")

# Funkcija za kreiranje Zero-shot prompta (BEZ IRRELEVANT)
def create_zero_shot_prompt(text, entity):
    return f"""Classify the sentiment of the following tweet towards the entity '{entity}'.
Options: Positive, Negative, Neutral.
Note: If the tweet is irrelevant to the entity or contains no clear sentiment, classify it as Neutral.

Tweet: {text}
Sentiment:"""

# Pokrećemo predikciju
y_pred_zero_shot = []

for index, row in tqdm(val_df.iterrows(), total=val_df.shape[0], desc="Zero-shot 3-Class"):
    prompt = create_zero_shot_prompt(row['text'], row['entity'])
    response = query_llm(prompt)
    y_pred_zero_shot.append(response)

# Čišćenje izlaza
def clean_prediction(pred):
    pred = pred.lower()
    if "positive" in pred: return "Positive"
    if "negative" in pred: return "Negative"
    # Sve ostalo (Neutral, Irrelevant, greške) ide u Neutral
    return "Neutral"

y_pred_zero_shot_clean = [clean_prediction(p) for p in y_pred_zero_shot]

# Evaluacija
print("\nRezultati Zero-shot pristupa (3 KLASE):")
print(classification_report(y_val, y_pred_zero_shot_clean))
print("Accuracy:", accuracy_score(y_val, y_pred_zero_shot_clean))

# Matrica konfuzije
plt.figure(figsize=(6, 5))
cm = confusion_matrix(y_val, y_pred_zero_shot_clean, labels=['Positive', 'Negative', 'Neutral'])
sns.heatmap(cm, annot=True, fmt='d', cmap='Oranges',
            xticklabels=['Pos', 'Neg', 'Neu'],
            yticklabels=['Pos', 'Neg', 'Neu'])
plt.title('Confusion Matrix (Zero-shot LLM)')
plt.show()

Tačnost je ostala ista (~57%) zato što izbacivanje reči "Irrelevant" iz prompta nije promenilo fundamentalno razumevanje modela.

Model i dalje ne prepoznaje neutralne tvitove (Recall mu je ostao nizak, oko 0.39). Iako sam rekao da "sve ostalo stavi u Neutral", model ima urođenu pristrasnost (bias) da u skoro svakom tekstu traži neki sentiment (pozitivan ili negativan), umesto da ga označi kao neutralan. To je tipično za instruction-tuned modele koji žele da budu "korisni" i daju konkretan odgovor.

Ukratko: Nije problem u formatu prompta, već u tome što modelu fali kontekst (primeri) šta tačno za tebe znači "Neutralno" u gejming žargonu. Zato ćemo implementirati Few-shot.

# **5. Pristup 3: Few-shot Klasifikacija**

Ovde biramo nekoliko primera iz trening seta i ubacujemo ih u prompt kako bismo modelu "pokazali" šta želimo.

In [ ]:
print("--- 3. FEW-SHOT PRISTUP ---")

# Biramo po jedan primer za svaku klasu iz trening seta da budu "demonstracija"
examples = train_df.groupby('sentiment').apply(lambda x: x.sample(1)).reset_index(drop=True)

few_shot_context = ""
for _, row in examples.iterrows():
    few_shot_context += f"Tweet: {row['text']}\nEntity: {row['entity']}\nSentiment: {row['sentiment']}\n\n"

print("Primeri koji se koriste u promptu:\n", few_shot_context)

def create_few_shot_prompt(text, entity):
    return f"""Classify the sentiment of the tweet towards the entity.
Options: Positive, Negative, Neutral, Irrelevant.

Examples:
{few_shot_context}
Task:
Tweet: {text}
Entity: {entity}
Sentiment:"""

# Pokrećemo predikciju
y_pred_few_shot = []
for index, row in tqdm(val_df.iterrows(), total=val_df.shape[0], desc="Few-shot inference"):
    prompt = create_few_shot_prompt(row['text'], row['entity'])
    response = query_llm(prompt)
    y_pred_few_shot.append(response)

y_pred_few_shot_clean = [clean_prediction(p) for p in y_pred_few_shot]

# Evaluacija
print("\nRezultati Few-shot pristupa:")
print(classification_report(y_val, y_pred_few_shot_clean))
print("Accuracy:", accuracy_score(y_val, y_pred_few_shot_clean))

# **Obrazloženje koda i metodologije (4. Deo: Few-shot Pristup)**
## 1. Koncept "In-Context Learning" (Učenje iz konteksta)

U ovom koraku primenjujem Few-shot metodologiju. Za razliku od Zero-shot pristupa gde model "nagađa" pravila, ovde mu u samom promptu dajem demonstraciju zadatka.

Selekcija primera: Koristio sam train_df.groupby('sentiment').apply(...) da automatski izvučem po jedan nasumičan primer za svaku klasu iz trening seta.

Cilj: Želim da model vidi kako izgleda jedan "Pozitivan", jedan "Negativan" i jedan "Neutralan" tvit pre nego što pokuša da klasifikuje novi. Ovo mu pomaže da "kalibriše" svoj izlaz i razume stil pisanja u datasetu.

Konstrukcija Prompta: Prompt je sada znatno složeniji. Sastoji se iz tri dela:

Instrukcija: ("Classify the sentiment...")

Kontekst (Examples): Gde mu prikazujem izvučene primere u formatu Tweet -> Entity -> Sentiment.

Zadatak (Task): Novi tvit koji treba da reši.

## 2. Tehnička realizacija

Dinamika: Kod dinamički gradi string few_shot_context pre iteracije. To znači da svi tvitovi u validacionom setu dobijaju iste primere za učenje. Ovo osigurava konzistentnost eksperimenta (svi su imali istu "pomoć").

Napomena za upozorenje (Warning): Poruka DeprecationWarning koju je biblioteka izbacila odnosi se na buduće promene u pandas biblioteci i ne utiče na tačnost trenutnih rezultata.

INTERPRETACIJA REZULTATA (Accuracy: 58.3%)
Tvoj rezultat je skočio sa 56.6% (Zero-shot) na 58.3% (Few-shot). Evo kako da "prodaš" ovaj rezultat u radu, iako deluje kao mali pomak:

### 1. Trend poboljšanja (Proof of Concept)

 "Primenom Few-shot pristupa zabeležen je porast tačnosti od 1.7%. Iako deluje skromno, ovo potvrđuje hipotezu da LLM model reaguje na primere (In-Context Learning). Model je uspeo da iskoristi ponuđeni kontekst da bolje razume zadatak."

### 2. Zašto poboljšanje nije veće? (Kritička analiza)

 Ovo je najbitniji deo za visoku ocenu – moraš objasniti zašto nismo dobili 80%+:

Problem nasumičnih primera: U ovom kodu smo koristili random primere (sample(1)).

Rizik: Moguće je da smo izvukli "lake" ili netipične primere koji nisu puno pomogli modelu da reši "teške" slučajeve u validacionom setu. (Da smo koristili one ručno birane primere iz "Pro" verzije koda, možda bi rezultat bio bolji).

Zadržavanje opcije "Irrelevant": U promptu je i dalje pisalo Options: ... Irrelevant. To zbunjuje model jer smo mi spojili klase u podatcima. Model vidi primer za "Neutral", ali mu mi nudimo i "Irrelevant", pa se on "cepa" između te dve opcije.

### 3. "Neutral" je i dalje problem (Recall: 0.37)

Negative Recall: 0.74 (Odlično prepoznaje hejt).

Positive Recall: 0.78 (Odlično prepoznaje pohvale).

Neutral Recall: 0.37 (Katastrofa).

### Zaključak

 "Čak i sa Few-shot primerima, FLAN-T5 model pokazuje snažnu pristrasnost ka polaritetu (Polarity Bias). On ima tendenciju da u svakom tvitu 'traži emociju'. Ako neko napiše običnu činjenicu o igrici, model će često pokušati da to protumači kao pozitivno ili negativno, umesto kao neutralno. Ovo je poznato ograničenje instruction-tuned modela koji su trenirani da budu 'korisni' i daju mišljenje."

### FINALNI ZAKLJUČAK ZA POREĐENJE

1) Klasični ML (SVM + TF-IDF): ~91%

Pobednik. Specijalizovan, brz, razume žargon jer je video ceo trening set.

2) Few-shot LLM: ~58.3%

Sredina. Pokazuje inteligenciju, uči iz primera, ali nedovoljno da savlada specifičnosti domena bez finetuning-a.

3) Zero-shot LLM: ~56.6%

Baza. Koristi samo opšte znanje engleskog jezika.

**6. Poredjenje rezultata (Vizuelizacija i Tabela)**

Na kraju, ovaj blok koda pravi tabelu i grafikon da uporediš tačnost sva tri pristupa, što ti treba za zaključak u radu.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Prikupljanje accuracy skorova
acc_ml = accuracy_score(y_val, y_pred_ml)
acc_zero = accuracy_score(y_val, y_pred_zero_shot_clean)
acc_few = accuracy_score(y_val, y_pred_few_shot_clean)

results_df = pd.DataFrame({
    'Metod': ['Classical ML (LogReg)', 'Zero-shot (LLM)', 'Few-shot (LLM)'],
    'Accuracy': [acc_ml, acc_zero, acc_few]
})

print("\n--- FINALNO POREĐENJE ---")
print(results_df)

# Crtanje grafika
plt.figure(figsize=(8, 5))
sns.barplot(x='Metod', y='Accuracy', data=results_df, palette='viridis')
plt.title('Poređenje tačnosti modela')
plt.ylim(0, 1)
for index, row in results_df.iterrows():
    plt.text(index, row.Accuracy + 0.02, f"{row.Accuracy:.2f}", color='black', ha="center")
plt.show()

# Prikaz matrice konfuzije za najbolji model (verovatno ML ili Few-shot)
print("Confusion Matrix (ML pristup):")
cm = confusion_matrix(y_val, y_pred_ml, labels=['Positive', 'Negative', 'Neutral', 'Irrelevant'])
sns.heatmap(cm, annot=True, fmt='d', xticklabels=['Pos', 'Neg', 'Neu', 'Irr'], yticklabels=['Pos', 'Neg', 'Neu', 'Irr'])
plt.ylabel('Stvarno')
plt.xlabel('Predviđeno')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.metrics import accuracy_score, confusion_matrix

print("--- 6. FINALNA VIZUELIZACIJA I POREĐENJE ---")

# Računanje tačnosti
acc_ml = accuracy_score(y_val, y_pred) # SVM model
acc_zero = accuracy_score(y_val, y_pred_zero_shot_clean) # Zero-shot
acc_few = accuracy_score(y_val, y_pred_few_shot_clean)   # Few-shot

# Kreiranje tabele rezultata
results_df = pd.DataFrame({
    'Metod': ['Classical ML (SVM)', 'Zero-shot (LLM)', 'Few-shot (LLM)'],
    'Accuracy': [acc_ml, acc_zero, acc_few]
})

print("\nTABELA REZULTATA:")
print(results_df)

# --- GRAFIK 1: POREĐENJE TAČNOSTI ---
plt.figure(figsize=(10, 6))
# Koristimo 'hue' i 'legend=False' da bi izbegli FutureWarning, i viridis paletu za profesionalni izgled
sns.barplot(x='Metod', y='Accuracy', data=results_df, palette='viridis', hue='Metod', legend=False)

plt.title('Finalno poređenje tačnosti (Accuracy)', fontsize=15)
plt.ylim(0, 1.1) # Malo mesta iznad 1.0 za tekst
plt.ylabel('Tačnost', fontsize=12)
plt.xlabel('Pristup', fontsize=12)

# Ispisivanje tačnih procenata iznad stubića
for index, row in results_df.iterrows():
    plt.text(index, row.Accuracy + 0.02, f"{row.Accuracy*100:.1f}%",
             color='black', ha="center", fontweight='bold', fontsize=12)
plt.show()

# --- GRAFIK 2: MATRICA KONFUZIJE POBEDNIKA (SVM) ---
print(f"\nDetaljna analiza grešaka za najbolji model (SVM):")
labels = ['Positive', 'Negative', 'Neutral'] # Bitno: Samo 3 klase!

cm = confusion_matrix(y_val, y_pred, labels=labels)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=labels, yticklabels=labels, annot_kws={"size": 14})
plt.title('Confusion Matrix: SVM (Pobednik)', fontsize=15)
plt.ylabel('Stvarna klasa', fontsize=12)
plt.xlabel('Predviđena klasa', fontsize=12)
plt.show()

#ZAKLJUČAK
U ovom radu sprovedena je komparativna analiza tri pristupa klasifikaciji sentimenta na domenski specifičnim podacima (Twitter objave o video igrama): klasičnog mašinskog učenja (Supervised ML), Zero-shot i Few-shot primene velikih jezičkih modela (LLM).

Na osnovu sprovedenih eksperimenata i dobijenih rezultata, mogu se izvući sledeći ključni zaključci:

## 1. Superiornost specijalizovanog ML modela

 Klasičan pristup zasnovan na LinearSVC algoritmu i TF-IDF vektorizaciji ostvario je ubedljivo najbolji rezultat sa tačnošću od 91.2%. Ovaj rezultat pokazuje da, kada je dostupna velika količina označenih podataka za trening (preko 70.000 primera), "stariji" i računarski jeftiniji algoritmi mogu nadmašiti opšte generativne modele. Ključ uspeha ovog modela leži u:

Korišćenju trigrama (n-gram range 1-3) koji su omogućili prepoznavanje negacija ("not good") i specifičnih fraza.

Zadržavanju interpunkcije (!, ?) i "stop-reči" koje nose sentiment.

Sposobnosti SVM-a da pronađe jasnu granicu između klasa u visoko-dimenzionalnom prostoru.

## 2. Ograničenja "Out-of-the-Box" LLM pristupa

 Veliki jezički model (Flan-T5 Large) pokazao je znatno niže performanse u poređenju sa nadgledanim učenjem:

Zero-shot (56.6%) je potvrdio da opšti model poseduje bazično razumevanje sentimenta, ali ne uspeva da prepozna specifičan gejming žargon, sarkazam i implicitna značenja bez dodatnog konteksta.

Few-shot (58.3%) je doneo blago poboljšanje, potvrđujući sposobnost modela da uči iz konteksta (In-Context Learning). Ipak, ovaj rast nije bio dovoljan da ugrozi ML model.

Glavni izazov za LLM bila je Neutralna klasa, gde je model pokazao nisku osetljivost (Recall ~37-39%), često pogrešno klasifikujući objektivne tvitove kao pozitivne ili negativne usled inherentne pristrasnosti modela (Polarity Bias).

## 3. Odnos cene, brzine i tačnosti

 Eksperiment ukazuje na jasan kompromis (trade-off):

ML pristup zahteva ogroman napor u prikupljanju i označavanju podataka (skupo vreme ljudi), ali rezultira modelom koji je izuzetno brz, precizan i jeftin za izvršavanje.

LLM pristup ne zahteva trening podatke (može se koristiti odmah), ali je sporiji, računarski zahtevniji i manje precizan na specifičnim domenima bez finog podešavanja (fine-tuning).

## Finalna presuda

 Za zadatke klasifikacije teksta u uskim domenima (kao što je gejming), gde su dostupni istorijski podaci, klasičan ML (SVM) ostaje optimalno rešenje. S druge strane, LLM pristupi su pogodniji za "hladan start" (kada nemamo podatke) ili za složenije zadatke koji zahtevaju generisanje teksta i obrazloženja, a ne samo grubu klasifikaciju.